<a href="https://colab.research.google.com/github/pahwa1902/LearMachineLearning/blob/master/DisasterTweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import re
import tensorflow as tf
import tensorflow_hub as hub
from keras.layers import LSTM
from tensorflow.keras import layers
tf.random.set_random_seed(1)
import tensorflow_datasets as tfds
from gensim.models import KeyedVectors
import numpy as np
tf.random.set_random_seed(1)


In [0]:
! ls '/content/drive/My Drive/nlp-getting-started'

In [0]:
data_train = pd.read_csv('/content/drive/My Drive/nlp-getting-started/train.csv')

In [0]:
training_data = pd.DataFrame(data_train)

In [5]:
training_data.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [0]:
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'(\s)@\w+', u'', x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'#', u'', x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'http\S+', u'', x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'[?|$|.|!|=>|&|;|:)|+|@|*|-]',u'',x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'[0-9]',u'',x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'(\w+):',u'',x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'(end\>)',u'',x))
#training_data['text'] = training_data['text'].apply(lambda x : re.sub('\s+','',x))
#training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'((?!\n)\s+)',u'',x))
#training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'(\s+)',u'',x))

In [78]:
training_data.iloc[1190]['text']

'Leicester_Merc  ICYMI  Ashes  Australia collapse at Trent Bridge  how Twitter rea\x89Û_  '

In [0]:
embedding = "https://tfhub.dev/google/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [0]:
summ=[]
for i in range(len(training_data)):
  a = training_data['text'].iloc[i]
  summ+=a.split(' ')

In [8]:
print(set(summ))

{'', 'distance//', 'reckless', 'talking', "Rome's", 'xbox', 'Downing', "Hasn't", 'BUCS', "lamp'", 'Uh', 'Mix', 'Tic', 'AdamTuss', 'sacrifice', 'clicks', 'declared', 'fandom', 'Buyout', 'ways', 'cancel', '[/]', 'Charlotte', 'Lawsuit', 'Refugee', 'WHAG', 'theological', 'tie', 'MartinMJ', 'Newport', 'TroisRivieres', 'chest', 'original', 'syndrome', 'hailstorm', 'Mmmm', 'FurTrix', 'AoMS', 'subtlety', 'Bernard', 'covers', 'sifting', 'hours', 'Lorries', 'stats', 'dry', 'steep', 'july', 'Hao', 'KristinDavis', 'butt', "(\nY'all", 'truck', 'SHUT', 'Insane', 'THANK', '\nHomealone', 'manåÊarmed', "cost\x89Û_'", 'credited', 'COLLAB', 'goal', 'long', 'Co', 'Boat', 'himself', 'TV', 'DUST', 'chains', 'twister', 'Reward', 'Matter', 'OEM', 'structure', 'boring', 'Cleveland', 'tyre', '/his', 'putting', 'instagram', 'Primary', 'mountains\x89Û_', 'am', 'FinancialTimes', 'HURRICANE', 'Fireworks', 'caked', 'Lightning_Wolf_', 'Hostage', 'derail', 'Hamilton', 'JampampK', 'newyorkcity', 'electrocuted', 'RIZZO'

In [0]:
def char2int(b):
  vocab = len(set(b))
  dict_vocab = {}
  a = list(set(b))
  for i in range(vocab):
    dict_vocab[a[i]] = i
  return dict_vocab,vocab

In [0]:
#char2id = {u:i for i,u in char2int[0].items()}
training_data = training_data.reindex(columns=training_data.columns.tolist()+['text2int'])


In [0]:
encoder = char2int(summ)[0]
vocab_size = char2int(summ)[1]

In [0]:
training_data['text2int'] = training_data['text'].apply(lambda x : np.array([encoder[c] for c in x.split(' ')]))

In [24]:
training_data.head()

,id,keyword,location,text,target,text2int
0,1,NaN,NaN,Our Deeds are the Reason of this earthquake Ma...,1,"[19420, 14278, 17900, 14131, 17787, 12448, 110..."
1,4,NaN,NaN,Forest fire near La Ronge Sask Canada,1,"[13821, 10223, 4223, 19728, 10703, 9480, 4196]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[17250, 11852, 19521, 4648, 14594, 7712, 2972,..."
3,6,NaN,NaN,", people receive wildfires evacuation orders i...",1,"[5799, 8988, 2993, 10104, 21673, 20221, 7712, ..."
4,7,NaN,NaN,Just got sent this photo from Ruby Alaska as s...,1,"[19463, 7520, 9857, 11033, 9233, 11068, 16553,..."


In [0]:
training_data = training_data.reindex(columns=training_data.columns.tolist()+['length'])
training_data['length']=training_data['text2int'].apply(lambda x : len(x))
pad_size = max(training_data['length'])
training_data['text2int'] = training_data['text2int'].apply(lambda x : np.array(x).tolist() + np.zeros(pad_size-len(x)).tolist())

In [0]:
label = training_data.pop('target')
train_data = training_data.iloc[:5700]['text']
train_label = label[:5700]
validate_data = training_data.iloc[5700:]['text']
validate_label = label[5700:]

In [0]:
train_data_mod = training_data.iloc[:5700]['text2int']
train_label_mod = label[:5700]
validate_data_mod = training_data.iloc[5700:]['text2int']
validate_label_mod = label[5700:]

In [0]:
train_data_mod=train_data_mod.tolist()
validate_data_mod = validate_data_mod.tolist()

In [0]:
columns=[]
for i in range(54):
  columns.append('v'+str(i))

In [0]:
train_dataset_mod = tf.data.Dataset.from_tensor_slices(train_data_mod)

In [0]:
iterator = train_dataset_mod.make_one_shot_iterator()
next_element = iterator.get_next()

In [0]:
with tf.Session() as sess:
    for i in range(1):
        val = sess.run(next_element)
        print(val)

In [0]:
label_dataset_mod = tf.data.Dataset.from_tensor_slices((train_label_mod))

In [0]:
train_label_dataset = tf.data.Dataset.zip((train_dataset_mod,label_dataset_mod)).shuffle(1000).repeat(5).batch(32)

In [0]:
validate_dataset_mod = tf.data.Dataset.from_tensor_slices(validate_data_mod)

In [0]:
label_validate_mod = tf.data.Dataset.from_tensor_slices((validate_label_mod))

In [0]:
validate_label_dataset = tf.data.Dataset.zip((validate_dataset_mod,validate_dataset_mod)).batch(32)

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data,train_label))
#train_label = tf.data.Dataset.from_tensor_slices((label))
train_dataset = train_dataset.shuffle(len(train_data))
#train_label = train_label.shuffle(len(train_data)).batch(64)

validate_dataset = tf.data.Dataset.from_tensor_slices((validate_data,validate_label))


In [0]:
def encode(data,label):
  encoded_data = np.array([encoder[c] for c in data.split(' ')])
  return encoded_data,label

In [0]:
def encode_fn(text,label):
  return tf.py_function(encode,inp=[text,label],Tout=(tf.int32,tf.int32))

In [0]:
train_dataset = train_dataset.map(encode_fn)
validate_dataset = validate_dataset.map(encode_fn)

In [0]:
train_data_new = train_dataset.shuffle(1000).padded_batch(32,padded_shapes=([-1],[]))

In [0]:
validate_data_new = validate_dataset.padded_batch(32,padded_shapes=([-1],[]))

In [0]:
model = tf.keras.Sequential();
model.add(tf.keras.layers.Embedding(vocab_size,64));
model.add(tf.keras.layers.LSTM(8,tf.keras.regularizers.l2(0.001)));
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [180]:
history = model.fit(train_label_dataset,epochs=10,verbose=1,validation_data=validate_label_dataset)

Train on 891 steps, validate on 60 steps
Epoch 1/10
891/891 [==============================] - 88s 99ms/step - loss: 0.6847 - acc: 0.5775 - val_loss: 685.2410 - val_acc: 0.7486
Epoch 2/10
723/891 [=======================>......] - ETA: 15s - loss: 0.6810 - acc: 0.5785

KeyboardInterrupt: ignored